In [19]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# First import the modules
import aicons.definitions.aicon
import aicons.bayesbrainGPT.utility_function.utility_function
import aicons.bayesbrainGPT.decision_making.action_space

# Now reload them for development/testing
importlib.reload(aicons.definitions.aicon)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_function)
importlib.reload(aicons.bayesbrainGPT.decision_making.action_space)

# Import the classes after reloading
from aicons.definitions.aicon import AIcon

In [20]:
# Create the AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: marketing_aicon


### Defining Action Space

In [21]:
# Define action space for budget allocation
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

ActionSpace(dimensions=2, constraints=1)

In [12]:
# Then print the action space details
action_space = aicon.action_space
print("\nAction Space Details:")
print("-" * 50)
print(f"Number of dimensions: {len(action_space.dimensions)}")
print("\nDimensions:")
for dim in action_space.dimensions:
    print(f"\nName: {dim.name}")
    print(f"Type: {dim.dim_type}")
    if dim.dim_type == 'continuous':
        print(f"Range: [{dim.min_value}, {dim.max_value}]")
        print(f"Step: {dim.step}")
    elif dim.dim_type == 'discrete':
        print(f"Values: {dim.values}")
print("-" * 50)

# You can also sample some actions to see what they look like
print("\nSample Actions:")
print("-" * 50)
for _ in range(3):
    action = action_space.sample()
    print(f"Action: {action}")


Action Space Details:
--------------------------------------------------
Number of dimensions: 2

Dimensions:

Name: google_budget
Type: continuous
Range: [0.0, 1000.0]
Step: 100.0

Name: facebook_budget
Type: continuous
Range: [0.0, 1000.0]
Step: 100.0
--------------------------------------------------

Sample Actions:
--------------------------------------------------
Action: {'google_budget': 400.0, 'facebook_budget': 600.0}
Action: {'google_budget': 500.0, 'facebook_budget': 500.0}
Action: {'google_budget': 200.0, 'facebook_budget': 800.0}


### Defining Utility Function

In [ ]:
# Define utility function for ROI
aicon.define_utility_function(
    utility_type='weighted_sum',
    weights={'google': 0.6, 'facebook': 0.4}
)


### Defining State Rep State

In [ ]:
# Add state factors
aicon.add_state_factor(
    name='base_conversion_rate',
    factor_type='continuous',
    value=0.05,
    uncertainty=0.01
)

aicon.add_state_factor(
    name='primary_channel',
    factor_type='categorical',
    value='google',
    categories=['google', 'facebook'],
    probs=[0.7, 0.3]
)